In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv("train.csv")
campaign = pd.read_csv("campaign_data.csv")
coupon_item_mapping = pd.read_csv("coupon_item_mapping.csv")
cust_demo = pd.read_csv("customer_demographics.csv")
cust_trans = pd.read_csv("customer_transaction_data.csv")
item_data = pd.read_csv("item_data.csv")
test = pd.read_csv("test.csv")

In [3]:
item_data.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [4]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [5]:
campaign

,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13
5,22,X,16/09/13,18/10/13
6,18,X,10/08/13,04/10/13
7,19,Y,26/08/13,27/09/13
8,17,Y,29/07/13,30/08/13
9,16,Y,15/07/13,16/08/13


In [6]:
campaign["start_date"] = pd.to_datetime(campaign["start_date"],format='%d/%m/%y')
campaign["end_date"] = pd.to_datetime(campaign["end_date"],format='%d/%m/%y')

In [7]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [8]:
cust_demo.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [9]:
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [10]:
cust_trans["date"] = pd.to_datetime(cust_trans["date"],format="%Y-%m-%d")

In [11]:
items = pd.merge(coupon_item_mapping,item_data,on="item_id",how="left")
items.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,107,75,56,Local,Grocery
2,494,76,209,Established,Grocery
3,522,77,278,Established,Grocery
4,518,77,278,Established,Grocery


In [12]:
cust_trans = pd.merge(cust_trans,items[["item_id","brand","brand_type","category"]],on="item_id",how='left')
cust_trans.drop_duplicates(keep="first",inplace=True)

#only campaign data and items whose coupon id is available
cust_trans = cust_trans[cust_trans["date"]>=campaign["start_date"].min()]
cust_trans.fillna("Others",inplace=True)
cust_trans["day_of_shopping"] = cust_trans["date"].dt.day_name()

cust_trans = pd.merge(cust_trans,coupon_item_mapping,on=["item_id"],how="left")
cust_trans.dropna(subset=["coupon_id"],inplace=True)
cust_trans.drop(["coupon_id"],axis=1,inplace=True)
cust_trans.drop_duplicates(keep="first",inplace=True)

In [13]:
#adding more to train data 
cust_trans_new = pd.merge(cust_trans,coupon_item_mapping,on=["item_id"],how="left")
cust_trans_new  = cust_trans_new[cust_trans_new["coupon_discount"]!=0]

cust_trans_new = pd.merge(cust_trans_new,train[["coupon_id","campaign_id",\
                                              "customer_id"]],\
                          on=["coupon_id","customer_id"],how="left")


cust_trans_new = cust_trans_new[cust_trans_new["date"]>=campaign["start_date"].min()]

cust_trans_new.drop_duplicates(keep="first",inplace=True)
cust_trans_new.dropna(subset=["campaign_id"],inplace=True)
cust_trans_new["redemption_status"]=1
cust_trans_new = cust_trans_new [["campaign_id","coupon_id","customer_id","redemption_status"]]
train = pd.concat([train,cust_trans_new],ignore_index=True)
train["id"] = np.arange(0,len(train))

In [14]:
#creating a new table for brand by binning and merging with train and test
cust_trans_new = pd.merge(cust_trans,coupon_item_mapping,on=["item_id"],how="left")
cust_trans_new.dropna(subset = ["coupon_id"],inplace=True)
cust_trans_new.drop_duplicates(keep="first",inplace=True)

brand_type_code = pd.DataFrame(cust_trans_new["brand"].value_counts())
bins = [0,100,300,600,1000,1500,2000,3000,6000,10000,100000,9999999999]
labels = ["brand_typ1","brand_typ2","brand_typ3","brand_typ4","brand_typ5","brand_typ6",\
         "brand_typ7","brand_typ8","brand_typ9","brand_typ10","brand_typ11"]
brand_type_code["brand_type_code"] = pd.cut(brand_type_code["brand"],bins=bins,labels=labels)
brand_type_code["brand"] = brand_type_code.index
brand_type_code.index = np.arange(0,len(brand_type_code))
brand_type_code.head()

,brand,brand_type_code
0,56.0,brand_typ11
1,278.0,brand_typ10
2,1337.0,brand_typ10
3,602.0,brand_typ10
4,544.0,brand_typ10


In [15]:
items = pd.merge(items,brand_type_code,on="brand",how="left")

In [16]:
cust_trans = pd.merge(cust_trans,brand_type_code,on="brand",how="left")
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,day_of_shopping,brand_type_code
0,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10
1,2012-08-12,994,8800,1,71.24,-13.89,0.0,1134,Established,Grocery,Sunday,brand_typ8
2,2012-08-12,994,14017,1,53.07,0.00,0.0,56,Local,Grocery,Sunday,brand_typ11
3,2012-08-12,994,15451,1,124.31,0.00,0.0,714,Established,Grocery,Sunday,brand_typ10
4,2012-08-12,994,16598,2,91.90,-17.81,0.0,56,Local,Grocery,Sunday,brand_typ11


In [17]:
temp = cust_trans.copy()
temp["selling_price"] = temp["selling_price"]/temp["quantity"]
temp["other_discount"] = np.where(temp["other_discount"]<0,\
                                 1,\
                                 0)

temp["coupon_discount"] = np.where(temp["coupon_discount"]<0,\
                                 1,\
                                 0)

t = temp.groupby(["item_id"]).agg({\
                                   "customer_id":{\
                                                  "unique":lambda x:x.nunique(),
                                                  "count_cust":"count",
                                                 },
                                  "selling_price":{\
                                                  "mean_price":"mean",
                                                  "std_price":lambda x:x[~x.isna()].std(),
                                                  },
                                   "other_discount":"sum",
                                   "coupon_discount":"sum",
                                  })
t.columns = ["n_customers_unique_item","n_customers_item","mean_price_item_item",\
             "std_dev_item","n_other_dis_item","n_coup_dis_item"]
t.fillna(0,inplace=True)
t["frequency_cust_item"] = t["n_customers_item"]/t["n_customers_unique_item"]
t.head()
item_data_cust_trans = t.copy()

In [18]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(item_data_cust_trans[["n_customers_unique_item","n_customers_item",\
                                                    "mean_price_item_item","std_dev_item",\
                                                    "frequency_cust_item"]])
item_data_cust_trans["item_id_type"] = kmeans.labels_

In [19]:
item_data_cust_trans.head()

,n_customers_unique_item,n_customers_item,mean_price_item_item,std_dev_item,n_other_dis_item,n_coup_dis_item,frequency_cust_item,item_id_type
item_id,,,,,,,,
1,1,1,124.310,0.000000,0,0,1.0,1
4,1,1,54.850,0.000000,0,0,1.0,1
12,1,1,35.620,0.000000,1,0,1.0,1
16,1,2,92.435,12.848130,1,0,2.0,1
17,2,2,9.619,0.001414,0,0,1.0,1


In [20]:
cust_trans = pd.merge(cust_trans,item_data_cust_trans[["item_id_type"]],on="item_id",how="left")

In [21]:
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,day_of_shopping,brand_type_code,item_id_type
0,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10,1
1,2012-08-12,994,8800,1,71.24,-13.89,0.0,1134,Established,Grocery,Sunday,brand_typ8,1
2,2012-08-12,994,14017,1,53.07,0.00,0.0,56,Local,Grocery,Sunday,brand_typ11,1
3,2012-08-12,994,15451,1,124.31,0.00,0.0,714,Established,Grocery,Sunday,brand_typ10,1
4,2012-08-12,994,16598,2,91.90,-17.81,0.0,56,Local,Grocery,Sunday,brand_typ11,1


In [22]:
items = pd.merge(items,item_data_cust_trans,on="item_id",how="left")
items.head()

,coupon_id,item_id,brand,brand_type,category,brand_type_code,n_customers_unique_item,n_customers_item,mean_price_item_item,std_dev_item,n_other_dis_item,n_coup_dis_item,frequency_cust_item,item_id_type
0,105,37,56,Local,Grocery,brand_typ11,2.0,5.0,49.750000,3.943741,4.0,0.0,2.5,1.0
1,107,75,56,Local,Grocery,brand_typ11,4.0,24.0,35.652188,5.515107,7.0,0.0,6.0,1.0
2,494,76,209,Established,Grocery,brand_typ10,1.0,1.0,89.050000,0.000000,1.0,0.0,1.0,1.0
3,522,77,278,Established,Grocery,brand_typ10,2.0,2.0,70.350000,23.928493,1.0,0.0,1.0,1.0
4,518,77,278,Established,Grocery,brand_typ10,2.0,2.0,70.350000,23.928493,1.0,0.0,1.0,1.0


In [23]:
train.head()

,campaign_id,coupon_id,customer_id,id,redemption_status
0,13.0,27,1053,0,0
1,13.0,116,48,1,0
2,9.0,635,205,2,0
3,13.0,644,1050,3,0
4,8.0,1017,1489,4,0


In [24]:
campaign = campaign.sort_values(by="start_date")
campaign.index = np.arange(0,len(campaign))

In [25]:
campaign

,campaign_id,campaign_type,start_date,end_date
0,26,X,2012-08-12,2012-09-21
1,27,Y,2012-08-25,2012-10-27
2,28,Y,2012-09-16,2012-11-16
3,29,Y,2012-10-08,2012-11-30
4,30,X,2012-11-19,2013-01-04
5,1,Y,2012-12-12,2013-01-18
6,2,Y,2012-12-17,2013-01-18
7,3,Y,2012-12-22,2013-02-16
8,4,Y,2013-01-07,2013-02-08
9,5,Y,2013-01-12,2013-02-15


In [26]:
test["campaign_id"].unique()

array([22, 20, 25, 17, 18, 21, 23, 19, 16, 24])

In [27]:
campaign_id_data = []
for i in range(0,len(campaign)):
    c_start = campaign["start_date"].loc[i]
    c_end = campaign["end_date"].loc[i]
    c_data = cust_trans[(cust_trans["date"]>=c_start)&(cust_trans["date"]<=c_end)]
    c_data["campaign"] = campaign["campaign_id"].iloc[i]
    campaign_id_data.append(c_data)
    #print(campaign["campaign_id"].iloc[i])

camp = campaign_id_data[0]
for i in range(1,len(campaign_id_data)):
    camp = pd.concat([camp,campaign_id_data[i]],ignore_index=True)

camp = pd.merge(camp,coupon_item_mapping,on=["item_id"],how="left")
camp.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,day_of_shopping,brand_type_code,item_id_type,campaign,coupon_id
0,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10,1,26,6
1,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10,1,26,22
2,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10,1,26,31
3,2012-08-12,994,8800,1,71.24,-13.89,0.0,1134,Established,Grocery,Sunday,brand_typ8,1,26,913
4,2012-08-12,994,8800,1,71.24,-13.89,0.0,1134,Established,Grocery,Sunday,brand_typ8,1,26,914


In [28]:
camp["campaign"].unique()

array([26, 27, 28, 29, 30,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,
       13])

In [29]:
#no of previous campaing he is part of 
# t = camp.groupby(["customer_id"]).agg({\
#                                       "campaign":lambda x:x.nunique(),\
                                      
#                                       })
# t.head(100)

In [30]:
cust_trans_test_campaign = test["customer_id"].unique()

In [31]:
# campaign_id_11 = campaign_id_data[11].copy()
# campaign_id_11 = pd.merge(campaign_id_11,coupon_item_mapping,on="item_id",how="left")
# campaign_id_11.dropna(inplace=True)
# campaign_id_11 = campaign_id_11[campaign_id_11['customer_id'].isin(cust_trans_test_campaign)]
# campaign_id_11 = campaign_id_11[["campaign","coupon_id","customer_id","coupon_discount"]]
# campaign_id_11.index = np.arange(0,len(campaign_id_11))

# campaign_id_11.drop_duplicates(subset=["campaign","coupon_id","customer_id"],keep="first",inplace=True)
# campaign_id_11["coupon_discount"] = np.where(campaign_id_11["coupon_discount"]!=0,\
#                          1,\
#                          0)
# campaign_id_11["coupon_discount"] = campaign_id_11[campaign_id_11["coupon_discount"]==0]
# campaign_id_11["coupon_discount"].replace(11,0,inplace=True)
# campaign_id_11["id"] = np.arange(0,len(campaign_id_11))
# campaign_id_11.rename(columns={"coupon_discount":"redemption_status","campaign":"campaign_id"},inplace=True)
# campaign_id_11.head()

In [32]:
# campaign_id_12 = campaign_id_data[12].copy()
# campaign_id_12 = pd.merge(campaign_id_12,coupon_item_mapping,on="item_id",how="left")
# campaign_id_12.dropna(inplace=True)
# campaign_id_12 = campaign_id_12[campaign_id_12['customer_id'].isin(cust_trans_test_campaign)]
# campaign_id_12 = campaign_id_12[["campaign","coupon_id","customer_id","coupon_discount"]]
# campaign_id_12.index = np.arange(0,len(campaign_id_12))

# campaign_id_12.drop_duplicates(subset=["campaign","coupon_id","customer_id"],keep="first",inplace=True)
# campaign_id_12["coupon_discount"] = np.where(campaign_id_12["coupon_discount"]!=0,\
#                          1,\
#                          0)
# campaign_id_12["coupon_discount"] = campaign_id_12[campaign_id_12["coupon_discount"]==0]
# campaign_id_12["coupon_discount"].replace(12,0,inplace=True)
# campaign_id_12["id"] = np.arange(0,len(campaign_id_12))
# campaign_id_12.rename(columns={"coupon_discount":"redemption_status","campaign":"campaign_id"},inplace=True)
# campaign_id_12.head()

In [33]:
# campaign_id_13 = campaign_id_data[13].copy()
# campaign_id_13 = pd.merge(campaign_id_13,coupon_item_mapping,on="item_id",how="left")
# campaign_id_13.dropna(inplace=True)
# campaign_id_13 = campaign_id_13[campaign_id_13['customer_id'].isin(cust_trans_test_campaign)]
# campaign_id_13 = campaign_id_13[["campaign","coupon_id","customer_id","coupon_discount"]]
# campaign_id_13.index = np.arange(0,len(campaign_id_13))

# campaign_id_13.drop_duplicates(subset=["campaign","coupon_id","customer_id"],keep="first",inplace=True)
# campaign_id_13["coupon_discount"] = np.where(campaign_id_13["coupon_discount"]!=0,\
#                          1,\
#                          0)
# campaign_id_13["coupon_discount"] = campaign_id_13[campaign_id_13["coupon_discount"]==0]
# campaign_id_13["coupon_discount"].replace(10,0,inplace=True)
# campaign_id_13["id"] = np.arange(0,len(campaign_id_13))
# campaign_id_13.rename(columns={"coupon_discount":"redemption_status","campaign":"campaign_id"},inplace=True)
# campaign_id_13.head()

In [34]:
# train = pd.concat([train,campaign_id_11],ignore_index=True)
# train = pd.concat([train,campaign_id_12],ignore_index=True)
# train = pd.concat([train,campaign_id_13],ignore_index=True)
# train.drop_duplicates(subset=["campaign_id","coupon_id","customer_id"],keep="first",inplace=True)
# train["id"] = np.arange(0,len(train))
# train.head()

In [35]:
t = items.groupby(["coupon_id","item_id_type"]).size().unstack(fill_value=0)
t.columns = ["n_item_type_id_in_coupon_" +str(col) for col in t.columns]

train = pd.merge(train,t,on="coupon_id",how="left")
test = pd.merge(test,t,on="coupon_id",how="left")

In [36]:
#no of times customer has been targeted
t = train.groupby("customer_id").agg({\
                                     "campaign_id":lambda x:x.nunique(),\
                                      "coupon_id":lambda x:x.nunique(),
                                     })
t.columns = ["no_of_campa_cust","no_of_coupon_cust"]
train = pd.merge(train,t,on="customer_id",how="left")

#No of coupon_ customers has been tragetd with
t = test.groupby("customer_id").agg({\
                                     "campaign_id":lambda x:x.nunique(),\
                                      "coupon_id":lambda x:x.nunique(),
                                     })
t.columns = ["no_of_campa_cust","no_of_coupon_cust"]
test = pd.merge(test,t,on="customer_id",how="left")

In [37]:
train.head()

,campaign_id,coupon_id,customer_id,id,redemption_status,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,no_of_campa_cust,no_of_coupon_cust
0,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30
1,13.0,116,48,1,0,0.0,2.0,0.0,0.0,0.0,1.0,15
2,9.0,635,205,2,0,33.0,11.0,0.0,0.0,11.0,5.0,76
3,13.0,644,1050,3,0,0.0,3.0,0.0,0.0,0.0,2.0,52
4,8.0,1017,1489,4,0,21.0,7.0,0.0,1.0,0.0,3.0,48


In [38]:
#no of unique customers in each campign
# t = train.groupby(["campaign_id"]).agg({\
#                                      "customer_id":lambda x:x.nunique(),
#                                      }).reset_index()
# t.columns = ["campaign_id","no_uniq_cust_camp_coup"]
# t.head()

# train = pd.merge(train,t,on="campaign_id",how="left")
# test = pd.merge(test,t,on="campaign_id",how="left")

In [39]:
#no of times customer has been targeted

In [40]:
#inserting campaign data into train and test

In [41]:
# c1 = ["Packaged Meat","Seafood","Dairy, Juices & Snacks","Prepared Food","Meat","Salads","Vegetables (cut)","Bakery"]
# c2 = ["Miscellaneous","Travel","Flowers & Plants",'Alcohol','Garden','Restauarant',"Fuel"]
# c3 = ["Natural Products","Pharmaceutical","Skin & Hair Care"]
# item_data["category"] = np.where(item_data["category"].isin(c1),\
#                                 "Food",\
#                                 item_data["category"])

# item_data["category"] = np.where(item_data["category"].isin(c2),\
#                                 "Others",\
#                                 item_data["category"])

# item_data["category"] = np.where(item_data["category"].isin(c3),\
#                                 "Health_Care",\
#                                 item_data["category"])

In [42]:
train = pd.merge(train,coupon_item_mapping,on="coupon_id",how="left")
train = pd.merge(train,item_data,on="item_id",how="left")
train = pd.merge(train,campaign,on="campaign_id",how="left")
train = pd.merge(train,brand_type_code,on="brand",how="left")

In [43]:
test = pd.merge(test,coupon_item_mapping,on="coupon_id",how="left")
test = pd.merge(test,item_data,on="item_id",how="left")
test = pd.merge(test,campaign,on="campaign_id",how="left")
test = pd.merge(test,brand_type_code,on="brand",how="left")

In [44]:
train.head()

,campaign_id,coupon_id,customer_id,id,redemption_status,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,no_of_campa_cust,no_of_coupon_cust,item_id,brand,brand_type,category,campaign_type,start_date,end_date,brand_type_code
0,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,24775,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9
1,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,14958,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9
2,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,40431,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9
3,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,20749,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9
4,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,56860,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9


In [45]:
t1 = train.groupby("campaign_id").agg({\
                                     "coupon_id":lambda x: x.nunique(),
                                      
                                      "customer_id": lambda x: x.nunique(),
                                      
                                      "item_id": lambda x: x.nunique(),
                                      
                                      "brand": lambda x: x.nunique(),
                                       
                                       "brand_type_code":lambda x:x.nunique()
                                       
                                     })
t1.columns = ["n_coupon_campaign","n_customer_campaign","n_item_campaign","n_brand_campaign","n_brand_type_campaign"]
#t1.head()

t2 = train[["campaign_id","brand","brand_type"]]
t2.drop_duplicates(subset=["campaign_id","brand","brand_type"],keep="first",inplace=True)
t2 = t2.groupby("campaign_id").agg({\
                                     "brand_type":{\
                                                "est_brand":lambda x: len(x[x=="Established"]),
                                                 "loc_brand":lambda x: len(x[x=="Local"])
                                                              }
                                   })

t2.columns = ["n_est_brand_campaign","n_loc_brand_campaign"]
t2 = t2.reset_index()
#t2.head()

t3 = train[["campaign_id","item_id","category"]]
t3.drop_duplicates(subset=["campaign_id","item_id","category"],keep="first",inplace=True)
t3 = t3.groupby(["campaign_id","category"]).size().unstack(fill_value=0)
t3.columns = ["n_campaign_"+col for col in t3.columns]
#t3.head()

t0 = train[["campaign_id","brand_type_code","brand_type"]]
t0.drop_duplicates(subset=["campaign_id","brand_type","brand_type_code"],keep="first",inplace=True)
t0= t0.groupby(["campaign_id","brand_type_code"]).size().unstack(fill_value=0)
t0.columns = ["n_campaign_"+col for col in t0.columns]


t4 = train[["campaign_id","coupon_id","n_item_type_id_in_coupon_0.0",\
           "n_item_type_id_in_coupon_1.0","n_item_type_id_in_coupon_2.0",\
           "n_item_type_id_in_coupon_3.0","n_item_type_id_in_coupon_4.0"]]
t4.drop_duplicates(subset=["campaign_id","coupon_id"],keep="first",inplace=True)
t4 = t4.groupby(["campaign_id"]).agg({\
                                      "n_item_type_id_in_coupon_0.0":"sum",
                                      "n_item_type_id_in_coupon_1.0":"sum",
                                      "n_item_type_id_in_coupon_2.0":"sum",
                                      "n_item_type_id_in_coupon_3.0":"sum",
                                      "n_item_type_id_in_coupon_4.0":"sum",
                                     })
t4.columns = ["n_campaign_"+col for col in t4.columns]



train = pd.merge(train,t0,on="campaign_id",how="left")
train = pd.merge(train,t1,on="campaign_id",how="left")
train = pd.merge(train,t2,on="campaign_id",how="left")
train = pd.merge(train,t3,on="campaign_id",how="left")
# train = pd.merge(train,t4,on="campaign_id",how="left")

In [46]:
t1 = test.groupby("campaign_id").agg({\
                                     "coupon_id":lambda x: x.nunique(),
                                      
                                      "customer_id": lambda x: x.nunique(),
                                      
                                      "item_id": lambda x: x.nunique(),
                                      
                                      "brand": lambda x: x.nunique(),
                                      
                                      "brand_type_code":lambda x:x.nunique()
                                       
                                     })
t1.columns = ["n_coupon_campaign","n_customer_campaign","n_item_campaign","n_brand_campaign","n_brand_type_campaign"]
#t1.head()

t2 = test[["campaign_id","brand","brand_type"]]
t2.drop_duplicates(subset=["campaign_id","brand","brand_type"],keep="first",inplace=True)
t2 = t2.groupby("campaign_id").agg({\
                                     "brand_type":{\
                                                "est_brand":lambda x: len(x[x=="Established"]),
                                                 "loc_brand":lambda x: len(x[x=="Local"])
                                                              }
                                   })
t2.columns = ["n_est_brand_campaign","n_loc_brand_campaign"]
t2 = t2.reset_index()
#t2.head()

t3 = test[["campaign_id","item_id","category"]]
t3.drop_duplicates(subset=["campaign_id","item_id","category"],keep="first",inplace=True)
t3 = t3.groupby(["campaign_id","category"]).size().unstack(fill_value=0)
t3.columns = ["n_campaign_"+col for col in t3.columns]
#t3.head()

t0 = test[["campaign_id","brand_type_code","brand_type"]]
t0.drop_duplicates(subset=["campaign_id","brand_type","brand_type_code"],keep="first",inplace=True)
t0= t0.groupby(["campaign_id","brand_type_code"]).size().unstack(fill_value=0)
t0.columns = ["n_campaign_"+col for col in t0.columns]



t4 = test[["campaign_id","coupon_id","n_item_type_id_in_coupon_0.0",\
           "n_item_type_id_in_coupon_1.0","n_item_type_id_in_coupon_2.0",\
           "n_item_type_id_in_coupon_3.0","n_item_type_id_in_coupon_4.0"]]
t4.drop_duplicates(subset=["campaign_id","coupon_id"],keep="first",inplace=True)
t4 = t4.groupby(["campaign_id"]).agg({\
                                      "n_item_type_id_in_coupon_0.0":"sum",
                                      "n_item_type_id_in_coupon_1.0":"sum",
                                      "n_item_type_id_in_coupon_2.0":"sum",
                                      "n_item_type_id_in_coupon_3.0":"sum",
                                      "n_item_type_id_in_coupon_4.0":"sum",
                                     })
t4.columns = ["n_campaign_"+col for col in t4.columns]

test = pd.merge(test,t0,on="campaign_id",how="left")
test = pd.merge(test,t1,on="campaign_id",how="left")
test = pd.merge(test,t2,on="campaign_id",how="left")
test = pd.merge(test,t3,on="campaign_id",how="left")
# test = pd.merge(test,t4,on="campaign_id",how="left")

In [47]:
#coupon data
t0 = items.groupby(["coupon_id","brand_type_code"]).size().unstack(fill_value=0)
t0.columns = ["n_coupon_"+col for col in t0.columns]

t1 = items.groupby("coupon_id").agg({\
                   
                  "item_id":lambda x: x.nunique(),
                  "brand":lambda x: x.nunique(),
                  }).reset_index()
t1.columns = ["coupon_id","n_coupon_items","n_coupon_brands"]

t2 = items.groupby(["coupon_id","brand_type"]).size().unstack(fill_value=0)
t2.columns = ["n_coupon_Esatb","n_coupon_Local"]


t3 = items.groupby(["coupon_id","category"]).size().unstack(fill_value = 0)
t3.columns = ["n_coupon"+ col for col in t3.columns]

In [48]:
train = pd.merge(train,t0,on="coupon_id",how="left")
train = pd.merge(train,t1,on="coupon_id",how="left")
train = pd.merge(train,t2,on="coupon_id",how="left")
train = pd.merge(train,t3,on="coupon_id",how="left")

test = pd.merge(test,t0,on="coupon_id",how="left")
test = pd.merge(test,t1,on="coupon_id",how="left")
test = pd.merge(test,t2,on="coupon_id",how="left")
test = pd.merge(test,t3,on="coupon_id",how="left")

In [49]:
train.drop_duplicates(subset = ["id"],keep="first",inplace=True)
test.drop_duplicates(subset = ["id"],keep="first",inplace=True)

In [50]:
train.head()

,campaign_id,coupon_id,customer_id,id,redemption_status,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,no_of_campa_cust,no_of_coupon_cust,item_id,brand,brand_type,category,campaign_type,start_date,end_date,brand_type_code,n_campaign_brand_typ1,n_campaign_brand_typ2,n_campaign_brand_typ3,n_campaign_brand_typ4,n_campaign_brand_typ5,n_campaign_brand_typ6,n_campaign_brand_typ7,n_campaign_brand_typ8,n_campaign_brand_typ9,n_campaign_brand_typ10,n_campaign_brand_typ11,n_coupon_campaign,n_customer_campaign,n_item_campaign,n_brand_campaign,n_brand_type_campaign,n_est_brand_campaign,n_loc_brand_campaign,n_campaign_Bakery,"n_campaign_Dairy, Juices & Snacks",n_campaign_Flowers & Plants,n_campaign_Garden,n_campaign_Grocery,n_campaign_Meat,n_campaign_Miscellaneous,n_campaign_Natural Products,n_campaign_Packaged Meat,n_campaign_Pharmaceutical,n_campaign_Prepared Food,n_campaign_Restauarant,n_campaign_Salads,n_campaign_Seafood,n_campaign_Skin & Hair Care,n_campaign_Travel,n_campaign_Vegetables (cut),n_coupon_brand_typ1,n_coupon_brand_typ2,n_coupon_brand_typ3,n_coupon_brand_typ4,n_coupon_brand_typ5,n_coupon_brand_typ6,n_coupon_brand_typ7,n_coupon_brand_typ8,n_coupon_brand_typ9,n_coupon_brand_typ10,n_coupon_brand_typ11,n_coupon_items,n_coupon_brands,n_coupon_Esatb,n_coupon_Local,n_couponBakery,"n_couponDairy, Juices & Snacks",n_couponFlowers & Plants,n_couponGarden,n_couponGrocery,n_couponMeat,n_couponMiscellaneous,n_couponNatural Products,n_couponPackaged Meat,n_couponPharmaceutical,n_couponPrepared Food,n_couponRestauarant,n_couponSalads,n_couponSeafood,n_couponSkin & Hair Care,n_couponTravel,n_couponVegetables (cut)
0,13.0,27,1053,0,0,0.0,109.0,0.0,0.0,0.0,1.0,30,24775,1636,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ9,2,2,2,2,2,1,1,1,1,1,1,207,1077,29561,2378,11,2367,11,56,504,652,95,9205,2076,58,2262,1921,9662,115,0,40,722,2170,17,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,64.0,0.0,125,2,125,0,0,0,0,0,125,0,0,0,0,0,0,0,0,0,0,0,0
125,13.0,116,48,1,0,0.0,2.0,0.0,0.0,0.0,1.0,15,36721,56,Local,Grocery,X,2013-05-19,2013-07-05,brand_typ11,2,2,2,2,2,1,1,1,1,1,1,207,1077,29561,2378,11,2367,11,56,504,652,95,9205,2076,58,2262,1921,9662,115,0,40,722,2170,17,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,1,0,3,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0
128,9.0,635,205,2,0,33.0,11.0,0.0,0.0,11.0,5.0,76,4763,560,Established,Pharmaceutical,Y,2013-03-11,2013-04-12,brand_typ3,1,1,1,0,2,1,0,1,1,1,1,18,176,898,18,9,16,2,0,135,0,0,586,0,1,0,0,176,0,0,0,0,0,0,0,0.0,0.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67,1,67,0,0,0,0,0,0,0,0,0,0,67,0,0,0,0,0,0,0
195,13.0,644,1050,3,0,0.0,3.0,0.0,0.0,0.0,2.0,52,65962,611,Established,Grocery,X,2013-05-19,2013-07-05,brand_typ5,2,2,2,2,2,1,1,1,1,1,1,207,1077,29561,2378,11,2367,11,56,504,652,95,9205,2076,58,2262,1921,9662,115,0,40,722,2170,17,6,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1,4,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0
199,8.0,1017,1489,4,0,21.0,7.0,0.0,1.0,0.0,3.0,48,6538,1558,Established,Grocery,X,2013-02-16,2013-04-05,brand_typ9,2,2,1,2,2,1,1,1,1,1,1,208,1076,14591,1617,11,1609,8,3,511,642,94,6793,2031,53,1882,1906,457,38,0,18,133,13,10,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,32,1,32,0,0,0,0,0,32,0,0,0,0,0,0,0,0,0,0,0,0


In [51]:
#inserting cust_trans data to train and test
#****************************
#****************************
#****************************
#****************************
#****************************

In [52]:
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,day_of_shopping,brand_type_code,item_id_type
0,2012-08-12,994,5179,1,113.63,0.00,0.0,133,Established,Grocery,Sunday,brand_typ10,1
1,2012-08-12,994,8800,1,71.24,-13.89,0.0,1134,Established,Grocery,Sunday,brand_typ8,1
2,2012-08-12,994,14017,1,53.07,0.00,0.0,56,Local,Grocery,Sunday,brand_typ11,1
3,2012-08-12,994,15451,1,124.31,0.00,0.0,714,Established,Grocery,Sunday,brand_typ10,1
4,2012-08-12,994,16598,2,91.90,-17.81,0.0,56,Local,Grocery,Sunday,brand_typ11,1


In [53]:
cust_trans_item = cust_trans[["item_id","coupon_discount"]]
cust_trans_item["coupon_discount"] = np.where(cust_trans_item["coupon_discount"]==0,\
                                             0,
                                             1)
cust_trans_item = cust_trans_item.groupby(["item_id","coupon_discount"]).size().unstack(fill_value=0)
cust_trans_item.columns = ["coup_disc_nottaken","coup_disc_taken"]
cust_trans_item.head()

,coup_disc_nottaken,coup_disc_taken
item_id,,
1,1,0
4,1,0
12,1,0
16,2,0
17,2,0


In [54]:
coupon_item_mapping.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [55]:
coupon_item_redemption = pd.merge(coupon_item_mapping,cust_trans_item,on="item_id",how="left")

coupon_item_redemption = coupon_item_redemption.groupby(["coupon_id"]).agg({\
                                                                          "coup_disc_nottaken":"sum",
                                                                           "coup_disc_taken":"sum"
                                                                          }).reset_index()


coupon_item_redemption["prob_coupon_redemption"] = coupon_item_redemption["coup_disc_taken"]/\
                            (coupon_item_redemption["coup_disc_nottaken"]+coupon_item_redemption["coup_disc_taken"])

coupon_item_redemption.head()

,coupon_id,coup_disc_nottaken,coup_disc_taken,prob_coupon_redemption
0,1,708.0,22.0,0.030137
1,2,56.0,0.0,0.000000
2,3,152.0,14.0,0.084337
3,4,418.0,29.0,0.064877
4,5,25.0,0.0,0.000000


In [56]:
train = pd.merge(train,coupon_item_redemption,on="coupon_id",how="left")
test = pd.merge(test,coupon_item_redemption,on="coupon_id",how="left")

In [57]:
cust_trans["coupon_discount"] = np.where(cust_trans["coupon_discount"]!=0,\
                                        1,\
                                        cust_trans["coupon_discount"])
cust_trans["other_discount"] = np.where(cust_trans["other_discount"]!=0,\
                                        1,\
                                        cust_trans["other_discount"])

cust_trans["brand"] = np.where(cust_trans["brand"]=="Others",\
                              99999999,\
                              cust_trans["brand"])
cust_trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,brand,brand_type,category,day_of_shopping,brand_type_code,item_id_type
0,2012-08-12,994,5179,1,113.63,0.0,0.0,133,Established,Grocery,Sunday,brand_typ10,1
1,2012-08-12,994,8800,1,71.24,1.0,0.0,1134,Established,Grocery,Sunday,brand_typ8,1
2,2012-08-12,994,14017,1,53.07,0.0,0.0,56,Local,Grocery,Sunday,brand_typ11,1
3,2012-08-12,994,15451,1,124.31,0.0,0.0,714,Established,Grocery,Sunday,brand_typ10,1
4,2012-08-12,994,16598,2,91.90,1.0,0.0,56,Local,Grocery,Sunday,brand_typ11,1


In [58]:
#no of coupons redeemed brand wise
t0 = pd.pivot_table(cust_trans,values="coupon_discount",columns="brand_type_code",index="customer_id",aggfunc="sum")
t0.fillna(0,inplace=True)
t0.columns = t0.columns
t0.head()

brand_type_code,brand_typ1,brand_typ2,brand_typ3,brand_typ4,brand_typ5,brand_typ6,brand_typ7,brand_typ8,brand_typ9,brand_typ10,brand_typ11
customer_id,,,,,,,,,,,
1,0.0,0.0,1.0,5.0,3.0,4.0,3.0,1.0,11.0,8.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,5.0,11.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0


In [59]:
#no of brand s purchased
t1 = cust_trans[["customer_id","brand_type_code"]]
t1 = t1.groupby(["customer_id","brand_type_code"]).size().unstack(fill_value=0)
t1.columns = ["n_brand_purchase_cust"+col for col in t1.columns]
t1.head()

,n_brand_purchase_custbrand_typ1,n_brand_purchase_custbrand_typ2,n_brand_purchase_custbrand_typ3,n_brand_purchase_custbrand_typ4,n_brand_purchase_custbrand_typ5,n_brand_purchase_custbrand_typ6,n_brand_purchase_custbrand_typ7,n_brand_purchase_custbrand_typ8,n_brand_purchase_custbrand_typ9,n_brand_purchase_custbrand_typ10,n_brand_purchase_custbrand_typ11
customer_id,,,,,,,,,,,
1,4,2,10,19,20,13,9,35,51,154,22
2,3,9,10,5,2,15,11,9,19,45,34
3,1,5,3,0,5,16,9,17,46,132,29
4,2,0,2,3,3,4,6,7,6,16,14
5,19,37,17,12,5,14,15,15,17,123,64


In [60]:
df = round(100*t0/(t1+1),2)
df.columns = ["brand_purcahse_prob_cust"+col for col in df.columns]
df.fillna(0,inplace=True)
df.head()

,brand_purcahse_prob_custbrand_typ1,brand_purcahse_prob_custbrand_typ10,brand_purcahse_prob_custbrand_typ11,brand_purcahse_prob_custbrand_typ2,brand_purcahse_prob_custbrand_typ3,brand_purcahse_prob_custbrand_typ4,brand_purcahse_prob_custbrand_typ5,brand_purcahse_prob_custbrand_typ6,brand_purcahse_prob_custbrand_typ7,brand_purcahse_prob_custbrand_typ8,brand_purcahse_prob_custbrand_typ9,brand_purcahse_prob_custn_brand_purchase_custbrand_typ1,brand_purcahse_prob_custn_brand_purchase_custbrand_typ10,brand_purcahse_prob_custn_brand_purchase_custbrand_typ11,brand_purcahse_prob_custn_brand_purchase_custbrand_typ2,brand_purcahse_prob_custn_brand_purchase_custbrand_typ3,brand_purcahse_prob_custn_brand_purchase_custbrand_typ4,brand_purcahse_prob_custn_brand_purchase_custbrand_typ5,brand_purcahse_prob_custn_brand_purchase_custbrand_typ6,brand_purcahse_prob_custn_brand_purchase_custbrand_typ7,brand_purcahse_prob_custn_brand_purchase_custbrand_typ8,brand_purcahse_prob_custn_brand_purchase_custbrand_typ9
customer_id,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
train = pd.merge(train,df,on="customer_id",how="left")
test = pd.merge(test,df,on="customer_id",how="left")


train = pd.merge(train,t1,on="customer_id",how="left")
test = pd.merge(test,t1,on="customer_id",how="left")

In [62]:
#no of coupons in tthat day 
t0 = pd.pivot_table(cust_trans,values="coupon_discount",columns="day_of_shopping",index="customer_id",aggfunc="sum")
t0.fillna(0,inplace=True)
t0.columns = t0.columns
t0.head()

day_of_shopping,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
customer_id,,,,,,,
1,0.0,7.0,7.0,6.0,9.0,5.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,19.0,2.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [63]:
#no fo unique qweekday vists
t1 = cust_trans[["date","customer_id","day_of_shopping"]]
t1.drop_duplicates(keep="first",inplace=True)
t1 = t1.groupby(["customer_id","day_of_shopping"]).size().unstack(fill_value=0)
t1.head()

day_of_shopping,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
customer_id,,,,,,,
1,4,6,2,3,7,7,5
2,1,2,4,0,3,3,3
3,3,4,0,3,3,5,4
4,4,4,6,3,0,0,1
5,18,12,13,11,12,27,15


In [64]:
df = round(t0/(t1+1),2)
df.fillna(0,inplace=True)

In [65]:
df.head()

day_of_shopping,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
customer_id,,,,,,,
1,0.00,1.0,2.33,1.5,1.12,0.62,0.33
2,0.00,0.0,0.00,0.0,0.00,0.00,0.25
3,4.75,0.4,0.00,0.0,0.00,0.00,0.00
4,0.00,0.0,0.00,0.0,0.00,0.00,0.00
5,0.05,0.0,0.00,0.0,0.00,0.04,0.00


In [66]:
# train = pd.merge(train,df,on="customer_id",how="left")
# test = pd.merge(test,df,on="customer_id",how="left")

In [67]:
t1 = cust_trans[["customer_id","item_id_type","coupon_discount"]]
t1 = t1.groupby(["customer_id","item_id_type","coupon_discount"]).size().unstack(fill_value=0).reset_index()
t1.columns = ["customer_id","item_id_type","not_taken_disc_item_id_type","taken_disc_item_id_type"]
t1["prob_discount_item_id_cust"] = 100*t1["taken_disc_item_id_type"]/\
                                    (t1["taken_disc_item_id_type"]+t1["not_taken_disc_item_id_type"])

train = pd.merge(train,t1,on="customer_id",how="left")
test = pd.merge(test,t1,on="customer_id",how="left")
del(t1)

In [68]:
t1 = cust_trans[["customer_id","category","coupon_discount"]]
t1.fillna("Others",inplace=True)
t1 = t1.groupby(["customer_id","category","coupon_discount"]).size().unstack(fill_value=0).reset_index()
t1.columns = ["customer_id","category","taken_disc","not_taken_disc"]
t1["prob_discount_category_cust"] = 100*t1["taken_disc"]/(t1["taken_disc"]+t1["not_taken_disc"])

t1 = pd.pivot_table(t1,values = "prob_discount_category_cust",columns="category",index="customer_id")
t1.columns = ["prob_cust_"+col for col in t1.columns]
t1.fillna(0,inplace=True)

train = pd.merge(train,t1,on="customer_id",how="left")
test = pd.merge(test,t1,on="customer_id",how="left")
del(t1)

In [69]:
t1 = cust_trans[["customer_id","category","other_discount"]]
t1.fillna("Others",inplace=True)
t1 = t1.groupby(["customer_id","category","other_discount"]).size().unstack(fill_value=0).reset_index()
t1.columns = ["customer_id","category","taken_disc","not_taken_disc"]
t1["prob_other_discount_category_cust"] = 100*t1["taken_disc"]/(t1["taken_disc"]+t1["not_taken_disc"])

t1 = pd.pivot_table(t1,values = "prob_other_discount_category_cust",columns="category",index="customer_id")
t1.columns = ["prob_other_dis_cust_"+col for col in t1.columns]
t1.fillna(0,inplace=True)

train = pd.merge(train,t1,on="customer_id",how="left")
test = pd.merge(test,t1,on="customer_id",how="left")
del(t1)

In [70]:
#transactioms when he Had discounts
t1 = cust_trans[["customer_id","coupon_discount","brand_type","category"]]
t1 = t1[t1["coupon_discount"]!=0]
t1.drop(["coupon_discount"],axis=1,inplace=True)

t2 = t1.groupby(["customer_id","brand_type"]).size().unstack(fill_value=0)
t2.columns = ["coup_dis_cust_Estab_n","coup_dis_cust_Local_n"]
t2.fillna(0,inplace=True)

t3 = t1.groupby(["customer_id","category"]).size().unstack(fill_value=0)
t3.columns = t3.columns

t3.columns = ["coup_dis_n_"+col for col in t3.columns]
t3.fillna(0,inplace=True)

train = pd.merge(train,t2,on="customer_id",how="left")
train = pd.merge(train,t3,on="customer_id",how="left")

test = pd.merge(test,t2,on="customer_id",how="left")
test = pd.merge(test,t3,on="customer_id",how="left")

In [71]:
#transactioms when he Had NO discounts
t1 = cust_trans[["customer_id","coupon_discount","brand_type","category"]]
t1 = t1[t1["coupon_discount"]==0]
t1.drop(["coupon_discount"],axis=1,inplace=True)

t2 = t1.groupby(["customer_id","brand_type"]).size().unstack(fill_value=0)
t2.columns = ["coup_dis_cust_Estab_n","coup_dis_cust_Local_n"]
t2.fillna(0,inplace=True)

t3 = t1.groupby(["customer_id","category"]).size().unstack(fill_value=0)
t3.columns = t3.columns

t3.columns = ["coup_no_dis_n_"+col for col in t3.columns]
t3.fillna(0,inplace=True)

train = pd.merge(train,t2,on="customer_id",how="left")
train = pd.merge(train,t3,on="customer_id",how="left")

test = pd.merge(test,t2,on="customer_id",how="left")
test = pd.merge(test,t3,on="customer_id",how="left")

In [72]:
train.dropna(subset=["brand_type_code"],inplace=True)

In [73]:
train.drop_duplicates(subset = ["id"],keep="first",inplace=True)
test.drop_duplicates(subset = ["id"],keep="first",inplace=True)

In [74]:
#probability of customers buying based on campaing type
# t = train.groupby(["customer_id","campaign_type","redemption_status"]).size().unstack(fill_value=0).reset_index()
# t.columns = ["customer_id","campaign_type","n_coup_dis_not_taken_camp_type","n_coup_dis_taken_camp_type"]
# t["total_campaign_type_XY"] = t["n_coup_dis_not_taken_camp_type"]+t["n_coup_dis_taken_camp_type"]
# t["probability_campaing_type_cust"] = 100*t["n_coup_dis_taken_camp_type"]/t["total_campaign_type_XY"]

# train = pd.merge(train,t[["customer_id","campaign_type","total_campaign_type_XY","probability_campaing_type_cust"]],\
#                  on=["customer_id","campaign_type"],how="left")
# test = pd.merge(test,t[["customer_id","campaign_type","total_campaign_type_XY","probability_campaing_type_cust"]],\
#                 on=["customer_id","campaign_type"],how="left")

# t.head()

In [75]:
for col in train.columns:
    if (int(train[col].isna().sum())>0):
        train[col].dtype != object
        train[col].fillna(0,inplace=True)

In [76]:
for col in test.columns:
    if (int(test[col].isna().sum())>0):
        test[col].dtype != object
        test[col].fillna(0,inplace=True)

In [77]:
train["train_test"] = "train"
train.head()

,campaign_id,coupon_id,customer_id,id,redemption_status,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,no_of_campa_cust,no_of_coupon_cust,item_id,brand,brand_type,category,campaign_type,start_date,end_date,brand_type_code,n_campaign_brand_typ1,n_campaign_brand_typ2,n_campaign_brand_typ3,n_campaign_brand_typ4,n_campaign_brand_typ5,n_campaign_brand_typ6,n_campaign_brand_typ7,n_campaign_brand_typ8,n_campaign_brand_typ9,n_campaign_brand_typ10,n_campaign_brand_typ11,n_coupon_campaign,n_customer_campaign,n_item_campaign,n_brand_campaign,n_brand_type_campaign,n_est_brand_campaign,n_loc_brand_campaign,n_campaign_Bakery,"n_campaign_Dairy, Juices & Snacks",n_campaign_Flowers & Plants,n_campaign_Garden,n_campaign_Grocery,n_campaign_Meat,n_campaign_Miscellaneous,n_campaign_Natural Products,n_campaign_Packaged Meat,n_campaign_Pharmaceutical,n_campaign_Prepared Food,n_campaign_Restauarant,n_campaign_Salads,n_campaign_Seafood,n_campaign_Skin & Hair Care,n_campaign_Travel,n_campaign_Vegetables (cut),n_coupon_brand_typ1,n_coupon_brand_typ2,n_coupon_brand_typ3,n_coupon_brand_typ4,n_coupon_brand_typ5,n_coupon_brand_typ6,n_coupon_brand_typ7,n_coupon_brand_typ8,n_coupon_brand_typ9,n_coupon_brand_typ10,n_coupon_brand_typ11,n_coupon_items,n_coupon_brands,n_coupon_Esatb,n_coupon_Local,n_couponBakery,"n_couponDairy, Juices & Snacks",n_couponFlowers & Plants,n_couponGarden,n_couponGrocery,n_couponMeat,n_couponMiscellaneous,n_couponNatural Products,n_couponPackaged Meat,n_couponPharmaceutical,n_couponPrepared Food,n_couponRestauarant,n_couponSalads,n_couponSeafood,n_couponSkin & Hair Care,n_couponTravel,n_couponVegetables (cut),coup_disc_nottaken,coup_disc_taken,prob_coupon_redemption,brand_purcahse_prob_custbrand_typ1,brand_purcahse_prob_custbrand_typ10,brand_purcahse_prob_custbrand_typ11,brand_purcahse_prob_custbrand_typ2,brand_purcahse_prob_custbrand_typ3,brand_purcahse_prob_custbrand_typ4,brand_purcahse_prob_custbrand_typ5,brand_purcahse_prob_custbrand_typ6,brand_purcahse_prob_custbrand_typ7,brand_purcahse_prob_custbrand_typ8,brand_purcahse_prob_custbrand_typ9,brand_purcahse_prob_custn_brand_purchase_custbrand_typ1,brand_purcahse_prob_custn_brand_purchase_custbrand_typ10,brand_purcahse_prob_custn_brand_purchase_custbrand_typ11,brand_purcahse_prob_custn_brand_purchase_custbrand_typ2,brand_purcahse_prob_custn_brand_purchase_custbrand_typ3,brand_purcahse_prob_custn_brand_purchase_custbrand_typ4,brand_purcahse_prob_custn_brand_purchase_custbrand_typ5,brand_purcahse_prob_custn_brand_purchase_custbrand_typ6,brand_purcahse_prob_custn_brand_purchase_custbrand_typ7,brand_purcahse_prob_custn_brand_purchase_custbrand_typ8,brand_purcahse_prob_custn_brand_purchase_custbrand_typ9,n_brand_purchase_custbrand_typ1,n_brand_purchase_custbrand_typ2,n_brand_purchase_custbrand_typ3,n_brand_purchase_custbrand_typ4,n_brand_purchase_custbrand_typ5,n_brand_purchase_custbrand_typ6,n_brand_purchase_custbrand_typ7,n_brand_purchase_custbrand_typ8,n_brand_purchase_custbrand_typ9,n_brand_purchase_custbrand_typ10,n_brand_purchase_custbrand_typ11,item_id_type,not_taken_disc_item_id_type,taken_disc_item_id_type,prob_discount_item_id_cust,prob_cust_Bakery,"prob_cust_Dairy, Juices & Snacks",prob_cust_Flowers & Plants,prob_cust_Garden,prob_cust_Grocery,prob_cust_Meat,prob_cust_Miscellaneous,prob_cust_Natural Products,prob_cust_Packaged Meat,prob_cust_Pharmaceutical,prob_cust_Prepared Food,prob_cust_Restauarant,prob_cust_Salads,prob_cust_Seafood,prob_cust_Skin & Hair Care,prob_cust_Travel,prob_cust_Vegetables (cut),prob_other_dis_cust_Bakery,"prob_other_dis_cust_Dairy, Juices & Snacks",prob_other_dis_cust_Flowers & Plants,prob_other_dis_cust_Garden,prob_other_dis_cust_Grocery,prob_other_dis_cust_Meat,prob_other_dis_cust_Miscellaneous,prob_other_dis_cust_Natural Products,prob_other_dis_cust_Packaged Meat,prob_other_dis_cust_Pharmaceutical,prob_other_dis_cust_Prepared Food,prob_other_dis_cust_

In [78]:
test["train_test"] = "test"
test.head()

,id,campaign_id,coupon_id,customer_id,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,no_of_campa_cust,no_of_coupon_cust,item_id,brand,brand_type,category,campaign_type,start_date,end_date,brand_type_code,n_campaign_brand_typ1,n_campaign_brand_typ2,n_campaign_brand_typ3,n_campaign_brand_typ4,n_campaign_brand_typ5,n_campaign_brand_typ6,n_campaign_brand_typ7,n_campaign_brand_typ8,n_campaign_brand_typ9,n_campaign_brand_typ10,n_campaign_brand_typ11,n_coupon_campaign,n_customer_campaign,n_item_campaign,n_brand_campaign,n_brand_type_campaign,n_est_brand_campaign,n_loc_brand_campaign,n_campaign_Bakery,"n_campaign_Dairy, Juices & Snacks",n_campaign_Flowers & Plants,n_campaign_Garden,n_campaign_Grocery,n_campaign_Meat,n_campaign_Miscellaneous,n_campaign_Natural Products,n_campaign_Packaged Meat,n_campaign_Pharmaceutical,n_campaign_Prepared Food,n_campaign_Salads,n_campaign_Seafood,n_campaign_Skin & Hair Care,n_campaign_Travel,n_campaign_Vegetables (cut),n_coupon_brand_typ1,n_coupon_brand_typ2,n_coupon_brand_typ3,n_coupon_brand_typ4,n_coupon_brand_typ5,n_coupon_brand_typ6,n_coupon_brand_typ7,n_coupon_brand_typ8,n_coupon_brand_typ9,n_coupon_brand_typ10,n_coupon_brand_typ11,n_coupon_items,n_coupon_brands,n_coupon_Esatb,n_coupon_Local,n_couponBakery,"n_couponDairy, Juices & Snacks",n_couponFlowers & Plants,n_couponGarden,n_couponGrocery,n_couponMeat,n_couponMiscellaneous,n_couponNatural Products,n_couponPackaged Meat,n_couponPharmaceutical,n_couponPrepared Food,n_couponRestauarant,n_couponSalads,n_couponSeafood,n_couponSkin & Hair Care,n_couponTravel,n_couponVegetables (cut),coup_disc_nottaken,coup_disc_taken,prob_coupon_redemption,brand_purcahse_prob_custbrand_typ1,brand_purcahse_prob_custbrand_typ10,brand_purcahse_prob_custbrand_typ11,brand_purcahse_prob_custbrand_typ2,brand_purcahse_prob_custbrand_typ3,brand_purcahse_prob_custbrand_typ4,brand_purcahse_prob_custbrand_typ5,brand_purcahse_prob_custbrand_typ6,brand_purcahse_prob_custbrand_typ7,brand_purcahse_prob_custbrand_typ8,brand_purcahse_prob_custbrand_typ9,brand_purcahse_prob_custn_brand_purchase_custbrand_typ1,brand_purcahse_prob_custn_brand_purchase_custbrand_typ10,brand_purcahse_prob_custn_brand_purchase_custbrand_typ11,brand_purcahse_prob_custn_brand_purchase_custbrand_typ2,brand_purcahse_prob_custn_brand_purchase_custbrand_typ3,brand_purcahse_prob_custn_brand_purchase_custbrand_typ4,brand_purcahse_prob_custn_brand_purchase_custbrand_typ5,brand_purcahse_prob_custn_brand_purchase_custbrand_typ6,brand_purcahse_prob_custn_brand_purchase_custbrand_typ7,brand_purcahse_prob_custn_brand_purchase_custbrand_typ8,brand_purcahse_prob_custn_brand_purchase_custbrand_typ9,n_brand_purchase_custbrand_typ1,n_brand_purchase_custbrand_typ2,n_brand_purchase_custbrand_typ3,n_brand_purchase_custbrand_typ4,n_brand_purchase_custbrand_typ5,n_brand_purchase_custbrand_typ6,n_brand_purchase_custbrand_typ7,n_brand_purchase_custbrand_typ8,n_brand_purchase_custbrand_typ9,n_brand_purchase_custbrand_typ10,n_brand_purchase_custbrand_typ11,item_id_type,not_taken_disc_item_id_type,taken_disc_item_id_type,prob_discount_item_id_cust,prob_cust_Bakery,"prob_cust_Dairy, Juices & Snacks",prob_cust_Flowers & Plants,prob_cust_Garden,prob_cust_Grocery,prob_cust_Meat,prob_cust_Miscellaneous,prob_cust_Natural Products,prob_cust_Packaged Meat,prob_cust_Pharmaceutical,prob_cust_Prepared Food,prob_cust_Restauarant,prob_cust_Salads,prob_cust_Seafood,prob_cust_Skin & Hair Care,prob_cust_Travel,prob_cust_Vegetables (cut),prob_other_dis_cust_Bakery,"prob_other_dis_cust_Dairy, Juices & Snacks",prob_other_dis_cust_Flowers & Plants,prob_other_dis_cust_Garden,prob_other_dis_cust_Grocery,prob_other_dis_cust_Meat,prob_other_dis_cust_Miscellaneous,prob_other_dis_cust_Natural Products,prob_other_dis_cust_Packaged Meat,prob_other_dis_cust_Pharmaceutical,prob_other_dis_cust_Prepared Food,prob_other_dis_cust_Restauarant,prob_other_dis_cust_Salads,pr

In [79]:
cust_demo["no_of_children"] = np.where(cust_demo["no_of_children"]=="3+",\
                                      3,\
                                      cust_demo["no_of_children"])

cust_demo["no_of_children"] = cust_demo["no_of_children"].astype(float)

cust_demo["marital_status"] = np.where(cust_demo["no_of_children"]>0,\
                                      "Married",\
                                      cust_demo["marital_status"])

age_married = ["46-55","36-45","70+","56-70","26-35"]
age_single = ["18-25"]
cust_demo["marital_status"] = np.where((cust_demo["marital_status"].isna())&\
                                       (cust_demo["age_range"].isin(age_married)),\
                                      "Married",\
                                      cust_demo["marital_status"])
cust_demo["marital_status"] = np.where((cust_demo["marital_status"].isna())&\
                                       (cust_demo["age_range"].isin(age_married)),\
                                      "Single",\
                                      cust_demo["marital_status"])

cust_demo["marital_status"] = np.where((cust_demo["marital_status"].isna())&\
                                       (~cust_demo["family_size"].isin([1,2,3])),\
                                      "Married",\
                                      cust_demo["marital_status"])
cust_demo["marital_status"] = np.where((cust_demo["marital_status"].isna())&\
                                       (cust_demo["family_size"].isin([1,2,3])),\
                                      "Single",\
                                      cust_demo["marital_status"])

cust_demo["family_size"] = np.where(cust_demo["family_size"]=="5+",\
                                      5,\
                                      cust_demo["family_size"])

cust_demo["family_size"] = cust_demo["family_size"].astype(float)
cust_demo["rented"] = cust_demo["rented"].astype(object)

In [80]:
train = pd.merge(train,cust_demo,on="customer_id",how="left")
test = pd.merge(test,cust_demo,on="customer_id",how="left")

In [81]:
train["marital_status"].fillna("Missing",inplace=True)
train["rented"].fillna(2,inplace=True)

test["marital_status"].fillna("Missing",inplace=True)
test["rented"].fillna(2,inplace=True)

In [82]:
train['days_campaign'] = (train["end_date"]-train["start_date"]).dt.days
test['days_campaign'] = (test["end_date"]-test["start_date"]).dt.days

# train["month_campaign"] = train["start_date"].dt.month
# test["month_campaign"] = test["start_date"].dt.month

# train['start_campaign_day'] = train["start_date"].dt.day_name()
# train['end_campaign_day'] = train["start_date"].dt.day_name()

# test["start_campaign_day"] = test["start_date"].dt.day_name()
# test["end_campaign_day"] = test["start_date"].dt.day_name()

In [83]:
train.index = train["id"]
test.index = test["id"]
col = ["id","campaign_id","coupon_id","item_id","brand_type","category",\
       "start_date","end_date","brand","brand_type_code","customer_id"]
train.drop(col,axis=1,inplace=True)
test.drop(col,axis=1,inplace=True)

In [84]:
X = train.drop(["redemption_status"],axis=1)
y = train["redemption_status"]

X["train_test"] = "train"
test["train_test"] ="test"
df = pd.concat([X,test])

In [85]:
set(X.columns)-set(test.columns)

{'n_campaign_Restauarant'}

In [86]:
X.drop(["n_campaign_Restauarant"],axis=1,inplace=True)

In [87]:

df["age_range"]  = np.where(df["age_range"]=="46-55",\
                              1,\
                              df["age_range"])
df["age_range"]  = np.where(df["age_range"]=="36-45",\
                              2,\
                              df["age_range"])
df["age_range"]  = np.where(df["age_range"]=="26-35",\
                              3,\
                              df["age_range"])
df["age_range"]  = np.where(df["age_range"]=="56-70",\
                              4,\
                              df["age_range"])
df["age_range"]  = np.where(df["age_range"]=="70+",\
                              5,\
                              df["age_range"])
df["age_range"]  = np.where(df["age_range"]=="18-25",\
                              6,\
                              df["age_range"])
df["age_range"].fillna(0,inplace=True)


In [88]:
categorical_features = ["campaign_type",
                        "marital_status","rented","age_range","family_size"]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[categorical_features] = df[categorical_features].apply(le.fit_transform)

In [89]:
df.head()

,age_range,brand_purcahse_prob_custbrand_typ1,brand_purcahse_prob_custbrand_typ10,brand_purcahse_prob_custbrand_typ11,brand_purcahse_prob_custbrand_typ2,brand_purcahse_prob_custbrand_typ3,brand_purcahse_prob_custbrand_typ4,brand_purcahse_prob_custbrand_typ5,brand_purcahse_prob_custbrand_typ6,brand_purcahse_prob_custbrand_typ7,brand_purcahse_prob_custbrand_typ8,brand_purcahse_prob_custbrand_typ9,brand_purcahse_prob_custn_brand_purchase_custbrand_typ1,brand_purcahse_prob_custn_brand_purchase_custbrand_typ10,brand_purcahse_prob_custn_brand_purchase_custbrand_typ11,brand_purcahse_prob_custn_brand_purchase_custbrand_typ2,brand_purcahse_prob_custn_brand_purchase_custbrand_typ3,brand_purcahse_prob_custn_brand_purchase_custbrand_typ4,brand_purcahse_prob_custn_brand_purchase_custbrand_typ5,brand_purcahse_prob_custn_brand_purchase_custbrand_typ6,brand_purcahse_prob_custn_brand_purchase_custbrand_typ7,brand_purcahse_prob_custn_brand_purchase_custbrand_typ8,brand_purcahse_prob_custn_brand_purchase_custbrand_typ9,campaign_type,coup_dis_cust_Estab_n_x,coup_dis_cust_Estab_n_y,coup_dis_cust_Local_n_x,coup_dis_cust_Local_n_y,coup_dis_n_Bakery,"coup_dis_n_Dairy, Juices & Snacks",coup_dis_n_Grocery,coup_dis_n_Meat,coup_dis_n_Miscellaneous,coup_dis_n_Natural Products,coup_dis_n_Packaged Meat,coup_dis_n_Pharmaceutical,coup_dis_n_Prepared Food,coup_dis_n_Seafood,coup_dis_n_Skin & Hair Care,coup_disc_nottaken,coup_disc_taken,coup_no_dis_n_Bakery,"coup_no_dis_n_Dairy, Juices & Snacks",coup_no_dis_n_Flowers & Plants,coup_no_dis_n_Garden,coup_no_dis_n_Grocery,coup_no_dis_n_Meat,coup_no_dis_n_Miscellaneous,coup_no_dis_n_Natural Products,coup_no_dis_n_Packaged Meat,coup_no_dis_n_Pharmaceutical,coup_no_dis_n_Prepared Food,coup_no_dis_n_Restauarant,coup_no_dis_n_Salads,coup_no_dis_n_Seafood,coup_no_dis_n_Skin & Hair Care,coup_no_dis_n_Travel,coup_no_dis_n_Vegetables (cut),days_campaign,family_size,income_bracket,item_id_type,marital_status,n_brand_campaign,n_brand_purchase_custbrand_typ1,n_brand_purchase_custbrand_typ10,n_brand_purchase_custbrand_typ11,n_brand_purchase_custbrand_typ2,n_brand_purchase_custbrand_typ3,n_brand_purchase_custbrand_typ4,n_brand_purchase_custbrand_typ5,n_brand_purchase_custbrand_typ6,n_brand_purchase_custbrand_typ7,n_brand_purchase_custbrand_typ8,n_brand_purchase_custbrand_typ9,n_brand_type_campaign,n_campaign_Bakery,"n_campaign_Dairy, Juices & Snacks",n_campaign_Flowers & Plants,n_campaign_Garden,n_campaign_Grocery,n_campaign_Meat,n_campaign_Miscellaneous,n_campaign_Natural Products,n_campaign_Packaged Meat,n_campaign_Pharmaceutical,n_campaign_Prepared Food,n_campaign_Restauarant,n_campaign_Salads,n_campaign_Seafood,n_campaign_Skin & Hair Care,n_campaign_Travel,n_campaign_Vegetables (cut),n_campaign_brand_typ1,n_campaign_brand_typ10,n_campaign_brand_typ11,n_campaign_brand_typ2,n_campaign_brand_typ3,n_campaign_brand_typ4,n_campaign_brand_typ5,n_campaign_brand_typ6,n_campaign_brand_typ7,n_campaign_brand_typ8,n_campaign_brand_typ9,n_couponBakery,"n_couponDairy, Juices & Snacks",n_couponFlowers & Plants,n_couponGarden,n_couponGrocery,n_couponMeat,n_couponMiscellaneous,n_couponNatural Products,n_couponPackaged Meat,n_couponPharmaceutical,n_couponPrepared Food,n_couponRestauarant,n_couponSalads,n_couponSeafood,n_couponSkin & Hair Care,n_couponTravel,n_couponVegetables (cut),n_coupon_Esatb,n_coupon_Local,n_coupon_brand_typ1,n_coupon_brand_typ10,n_coupon_brand_typ11,n_coupon_brand_typ2,n_coupon_brand_typ3,n_coupon_brand_typ4,n_coupon_brand_typ5,n_coupon_brand_typ6,n_coupon_brand_typ7,n_coupon_brand_typ8,n_coupon_brand_typ9,n_coupon_brands,n_coupon_campaign,n_coupon_items,n_customer_campaign,n_est_brand_campaign,n_item_campaign,n_item_type_id_in_coupon_0.0,n_item_type_id_in_coupon_1.0,n_item_type_id_in_coupon_2.0,n_item_type_id_in_coupon_3.0,n_item_type_id_in_coupon_4.0,n_loc_brand_campaign,no_of_campa_cust,no_of_children,no_of_coupon_cust,not_taken_disc_item_id_type,prob_coupon_redemption,prob_cust_Bakery,"prob_cust_Dairy, Juices & Snack

In [ ]:
#df.drop(["marital_status","rented","age_range","family_size"],axis=1,inplace=True)

In [91]:
X = df[df["train_test"]=="train"]
test = df[df["train_test"]=="test"]
X.drop(["train_test"],axis=1,inplace=True)
test.drop(["train_test"],axis=1,inplace=True)

In [92]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

In [93]:
categorical_features_idx = [X.columns.get_loc(c) for c in categorical_features]

lgb = LGBMClassifier(
                      boosting_type='goss',
                      objective='binary',
                      n_estimators=1000,
                      learning_rate=0.01,
    
                      colsample_bytree=0.9,
                      num_leaves=120,
                      min_data_in_leaf=7,
                      max_depth=7,
    
#                      bagging_fraction=0.9,
#                      bagging_freq=1,
#                      pos_bagging_fraction = 0.2,
#                      neg_bagging_fraction = 0.6,

                    categorical_features = categorical_features_idx,
                     )

In [94]:
scores = cross_val_score(lgb, X, y.values.ravel(),scoring='roc_auc', cv=5)
scores 

array([0.95619759, 0.96254372, 0.97536275, 0.98207497, 0.97864338])

In [97]:
temp = test.copy()
lgb  = lgb.fit(X,y)
pred_lgb = lgb.predict_proba(test)[:,-1]
temp["redemption_status"] = pred_lgb 
temp = temp[["redemption_status"]]
temp.to_csv("try56.csv")
temp.head()

,redemption_status
id,
3,0.250420
4,0.009975
5,0.212281
8,0.000286
10,0.143204


In [98]:
temp[temp["redemption_status"]>=0.5].shape

(304, 1)

In [ ]:
# X.head()

In [ ]:
# from catboost import CatBoostClassifier
# categorical_features_idx = [X.columns.get_loc(c) for c in categorical_features]
# cat=CatBoostClassifier(iterations=2000,  
#                          learning_rate=0.01,
                         
#                          loss_function='Logloss',
#                          custom_loss=['AUC'],
#                          cat_features = categorical_features_idx,
#                         verbose=False
#                         )

In [ ]:
# scores = cross_val_score(cat, X, y.values.ravel(),scoring='roc_auc', cv=5)
# scores 

In [ ]:
# cat  = cat.fit(X,y.values.ravel())
# pred_cat = cat.predict_proba(test)

In [ ]:
# from xgboost.sklearn import XGBClassifier

# xgb1 = XGBClassifier(
#  learning_rate =0.01,
#  n_estimators=2000,
#  max_depth=6,
#  objective= 'binary:logistic',
#  nthread=8,
#  seed=27)
